# 🐞 Bayesian-AI Debug Dashboard

**Objective:** Interactive system validation, debugging, and performance profiling.
**Sections:**
1. Environment Check
2. Data Pipeline Test
3. Core Component Tests
4. Mini Training Run (5 Iterations)
5. Probability Table Analysis
6. Performance Profiling
7. DOE Simulation Preview
8. Quick Fixes & Utilities

## 1. Environment Check 🖥️
Verify Python environment, CUDA availability, and key dependencies.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
from numba import cuda

# Add root to path
current_dir = Path(os.getcwd())
project_root = current_dir
while not (project_root / 'requirements.txt').exists():
    parent = project_root.parent
    if parent == project_root:
        break
    project_root = parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
print(f"Project Root: {project_root}")

from config.settings import OPERATIONAL_MODE, RAW_DATA_PATH

# 1. Check Operational Mode
print(f"\nOperational Mode: {OPERATIONAL_MODE}")

# 2. Check CUDA
try:
    if cuda.is_available():
        print(f"🟢 CUDA Available: {cuda.detect()}")
        device = cuda.get_current_device()
        print(f"   Device: {device.name}")
    else:
        print("🔴 CUDA Not Available. System running in CPU mode.")
except Exception as e:
    print(f"🔴 CUDA Check Failed: {e}")


## 2. Data Pipeline Test 🛀
Validate loading of a single data file (.dbn.zst or .parquet).

In [ ]:
from training.orchestrator import get_data_source
from tests.utils import get_test_data_files

try:
    files = get_test_data_files()
    if not files:
        print("🔴 No data files found.")
    else:
        test_file = files[0]
        print(f"Loading {os.path.basename(test_file)}...")
        df = get_data_source(test_file)
        print(f"🟢 Load Success! Shape: {df.shape}")
        print(df.head())
        
        # Store for later use
        sample_data = df.head(1000).copy() if len(df) > 1000 else df.copy()

except Exception as e:
    print(f"🔴 Pipeline Test Failed: {e}")

## 3. Core Component Tests ⚙️
Verify StateVector, BayesianBrain, LayerEngine, and CUDA modules.

In [ ]:
from core.state_vector import StateVector
from core.bayesian_brain import BayesianBrain
from core.layer_engine import LayerEngine

print("--- Component Status ---")

# 1. StateVector
try:
    sv = StateVector.null_state()
    assert hash(sv) is not None
    print("🟢 StateVector: Operational")
except Exception as e:
    print(f"🔴 StateVector: Failed ({e})")

# 2. BayesianBrain
try:
    bb = BayesianBrain()
    print("🟢 BayesianBrain: Initialized")
except Exception as e:
    print(f"🔴 BayesianBrain: Failed ({e})")

# 3. LayerEngine
try:
    le = LayerEngine(use_gpu=cuda.is_available() if 'cuda' in locals() else False)
    print(f"🟢 LayerEngine: Initialized (GPU={le.use_gpu})")
except Exception as e:
    print(f"🔴 LayerEngine: Failed ({e})")

## 4. Mini Training Run (5 Iterations) 🏃‍♂️
Interactive 5-iteration training on sample data.

In [ ]:
from training.orchestrator import TrainingOrchestrator

def run_mini_training(b):
    with out_area:
        clear_output()
        print("Initializing Mini Training (5 iterations)...")
        
        if 'sample_data' not in globals():
            print("🔴 Sample data not loaded. Run Section 2 first.")
            return

        try:
            # Determine GPU usage dynamically
            use_gpu = False
            try:
                use_gpu = cuda.is_available()
            except:
                pass
            
            orch = TrainingOrchestrator(
                asset_ticker='MNQ', 
                data=sample_data, 
                output_dir='debug_outputs/mini_run',
                use_gpu=use_gpu,
                verbose=chk_verbose.value
            )
            
            res = orch.run_training(iterations=5)
            print("\n🟢 Mini Run Complete!")
            print(f"Total Trades: {res['total_trades']}")
            print(f"PnL: ${res['pnl']:.2f}")
            print(f"Win Rate: {res['win_rate']:.1%}")
            
        except Exception as e:
            print(f"🔴 Training Failed: {e}")

chk_verbose = widgets.Checkbox(value=False, description='Verbose Mode')
btn_run = widgets.Button(description="Run 5 Iterations", button_style='primary')
out_area = widgets.Output()

btn_run.on_click(run_mini_training)
display(widgets.VBox([widgets.HBox([btn_run, chk_verbose]), out_area]))

## 5. Probability Table Analysis 📊
Analyze the learned probability tables.

In [ ]:
import pickle

def load_and_analyze_table(path):
    if not os.path.exists(path):
        print(f"File not found: {path}")
        return
    
    print(f"Loading {path}...")
    try:
        with open(path, 'rb') as f:
            data = pickle.load(f)
            
        # Handle both raw dict and BayesianBrain object
        table = data['table'] if isinstance(data, dict) and 'table' in data else getattr(data, 'table', {})
        
        print(f"Total States: {len(table)}")
        
        # Convert to DF
        records = []
        for state, stats in table.items():
            total = stats['total']
            wins = stats['wins']
            if total > 0:
                records.append({
                    'state_hash': hash(state),
                    'total': total,
                    'wins': wins,
                    'win_rate': wins/total,
                    'L1': state.L1_bias,
                    'L5': state.L5_trend
                })
        
        if records:
            df_stats = pd.DataFrame(records)
            print("\nTop 5 States by Sample Size:")
            display(df_stats.sort_values('total', ascending=False).head())
            
            fig = px.scatter(df_stats, x='total', y='win_rate', 
                             color='L1', title='Win Rate vs Sample Size',
                             hover_data=['L5'])
            fig.show()
        else:
            print("No valid states found.")
            
    except Exception as e:
        print(f"Error analyzing table: {e}")

btn_analyze = widgets.Button(description="Analyze Mini-Run Model")
btn_analyze.on_click(lambda b: load_and_analyze_table('debug_outputs/mini_run/probability_table.pkl'))
display(btn_analyze)

## 6. Performance Profiling ⏱️
Benchmark state hashing and component latencies.

In [ ]:
import timeit

print("Benchmarking StateVector Hashing...")
setup_code = "from core.state_vector import StateVector; sv = StateVector.null_state()"
t = timeit.timeit("hash(sv)", setup=setup_code, number=100000)
print(f"Create & Hash 100k states: {t:.4f}s")

## 7. DOE Simulation Preview 🧪
Visualize parameter combinations for future optimization.

In [ ]:
import itertools

param_grid = {
    'min_prob': [0.70, 0.75, 0.80, 0.85],
    'min_conf': [0.20, 0.30, 0.40],
    'stop_loss': [10, 20, 30],
    'kill_zones': ['tight', 'wide']
}
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(f"Total Parameter Combinations: {len(combinations)}")

print("Sample Combinations:")
for c in combinations[:5]:
    print(c)

## 8. Quick Fixes & Utilities 🛠️
Common maintenance tasks.

In [ ]:
import shutil

def clean_pycache():
    print("Cleaning __pycache__...")
    count = 0
    for root, dirs, files in os.walk('.'):
        for d in dirs:
            if d == '__pycache__':
                shutil.rmtree(os.path.join(root, d))
                count += 1
    print(f"Removed {count} directories.")

btn_clean = widgets.Button(description="Clear PyCache")
btn_clean.on_click(lambda b: clean_pycache())
display(btn_clean)